In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
%%time

import malaya
malaya

CPU times: user 2.82 s, sys: 2.63 s, total: 5.45 s
Wall time: 2.24 s


/home/ubuntu/dev/malaya/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3361
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/ubuntu/dev/malaya/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3879
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


<module 'malaya' from '/home/ubuntu/dev/malaya/malaya/__init__.py'>

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
sastrawi = malaya.stem.sastrawi()

In [5]:
# !wget https://github.com/huseinzol05/malay-dataset/blob/master/normalization/stemmer/stem.zip?raw=true
# !wget https://f000.backblazeb2.com/file/malay-dataset/wiki-stem.json
# !unzip stem.zip?raw=true

In [6]:
with open('/home/ubuntu/malaya/stemmer-data-v5.txt') as fopen:
    stemming = fopen.read().split('\n')
stemming[:5]

['spider\tspider',
 'transeksual\ttranseksual',
 'khusus\tkhusus',
 'kriminologi\tkriminologi',
 'negeripingat\tnegeripingat']

In [7]:
left, right = [], []

for s in stemming:
    l, r = s.split('\t')
    left.append(l)
    right.append(r)
    
len(left)

41528

In [8]:
import json

with open('/home/ubuntu/malaya/news-stem.json') as fopen:
    news = json.load(fopen)
    
for s in news:
    left.append(s[0])
    right.append(s[1])

In [9]:
with open('/home/ubuntu/malaya/wiki-stem.json') as fopen:
    wiki = json.load(fopen)
    
for s in wiki:
    left.append(s[0])
    right.append(s[1])

In [10]:
len(left), len(right)

(1832338, 1832338)

In [11]:
left[1], right[0]

('transeksual', 'spider')

In [12]:
sastrawi.stem(f'{left[1]}lah')

'transeksual'

In [13]:
stem = sastrawi.stem('sakitnya')
stem

'sakit'

In [14]:
hujung = {
    'kan': ['kann', 'kkan', 'kaann', 'kn', 'kkn'],
    'lah': ['lh', 'lhh', 'lhaa', 'lha', 'laah', 'laahh', 'ler'],
    'nya': ['nye', 'ny', 'nyee', 'nyye', 'nyo', 'nyoo'],
}

In [15]:
import re
import random

vowels = 'aeiou'
consonants = 'bcdfghjklmnpqrstvwxyz'

In [16]:
def augment_hubung(word, p_remove_vowels = 0.5, p_duplicate = 0.7, min_duplicate = 2, max_duplicate = 3):
    word = ['' if c in vowels and random.random() > p_remove_vowels else c for c in word]
    word = list(filter(None, word))
    for i in range(len(word)):
        w = word[i]
        if random.random() > p_duplicate:
            duplicate = random.randint(min_duplicate, max_duplicate)
            word[i] = w * duplicate
    return ''.join(word)

In [17]:
malaya.augmentation.vowel_alternate('sakit')

'sakt'

In [18]:
augment_hubung('ter')

'ter'

In [19]:
string = 'tersakitnya'
stemmed = sastrawi.stem(string)
index = [(m.start(0), m.end(0)) for m in re.finditer(stemmed, string, flags=re.IGNORECASE)][0]
permulaan = string[:index[0]]
hujungan = string[index[1]:]
permulaan, hujungan

('ter', 'nya')

In [20]:
awalan = ['pe', 'pem', 'pen', 'peng', 'pel', 'ke', 'juru', 'pra', 'dwi',
         'me', 'memper', 'ber', 'ter', 'di', 'diper', 'ter', 'te', 'se',
         'mem', 'diper']

akhiran = ['lah', 'an', 'kan', 'nya', 'wan', 'wati']

In [21]:
random.choice(hujung.get(hujungan, [hujungan]))

'nyee'

In [22]:
sastrawi.stem('memperhambakan')

'hamba'

In [23]:
set_left = set(left)

In [24]:
from tqdm import tqdm

In [25]:
new_left, new_right = [], []
for w in tqdm(malaya.dictionary.bahasa.dbp.words | malaya.dictionary.bahasa.cambridge_words.words | malaya.dictionary.bahasa.kamus_dewan.words | malaya.dictionary.bahasa.words.words):
    if w in set_left:
        continue
    s = sastrawi.stem(w)
    new_left.append(w)
    new_right.append(s)
    set_left.add(w)
    
left.extend(new_left)
right.extend(new_right)

100%|██████████████████████████████████████████████████████████████████████████████████| 63838/63838 [00:03<00:00, 17825.01it/s]


In [26]:
len(left), len(right)

(1855109, 1855109)

In [27]:
def augment(word, p_augment_word = 0.7):
    awal = random.choice(awalan)
    akhir = random.choice(akhiran)
    joined = awal + word + akhir
    stemmed = sastrawi.stem(joined)
    if stemmed == joined:
        return word, word
    
    index = [(m.start(0), m.end(0)) for m in re.finditer(stemmed, joined, flags=re.IGNORECASE)][0]
    permulaan = joined[:index[0]]
    hujungan = joined[index[1]:]
    
    if not len(permulaan) and not len(hujungan):
        return False, left, right
    
    print(joined, stemmed, permulaan, hujungan)
    
    augmented_permulaan = augment_hubung(permulaan)
    if len(augmented_permulaan) > 1:
        permulaan = augmented_permulaan
    
    augmented_hujungan = augment_hubung(hujungan)
    if len(augmented_hujungan) > 1:
        hujungan = augmented_hujungan
        
    if random.random() > p_augment_word:
        word = malaya.augmentation.vowel_alternate(word)
    
    return True, permulaan + word + hujungan, word

def augment_left_right(left, right, augment_hubungan = True, p_augment_word = 0.7):
    index = [(m.start(0), m.end(0)) for m in re.finditer(right, left, flags=re.IGNORECASE)]
    permulaan = left[:index[0][0]]
    hujungan = left[index[-1][1]:]

    not_right = left[index[0][0]:index[-1][1]]
    # print(permulaan, hujungan, not_right)
    
    if not len(permulaan) and not len(hujungan):
        return False, left, right
    
    if augment_hubungan:
        augmented_permulaan = augment_hubung(permulaan)
        if len(augmented_permulaan) > 1:
            permulaan = augmented_permulaan
    
    if augment_hubungan:
        augmented_hujungan = augment_hubung(hujungan)
        if len(augmented_hujungan) > 1:
            hujungan = augmented_hujungan
    
    if random.random() > p_augment_word and not_right == right:
        right = malaya.augmentation.vowel_alternate(right)
        selected = right
    else:
        selected = not_right
    
    joined = permulaan + selected + hujungan
    joined = re.sub(r'^-+|-+$', '-', joined).strip()
    
    
    if joined == left:
        result = False
    else:
        result = True
    
    return result, joined, right

In [28]:
left[:10]

['spider',
 'transeksual',
 'khusus',
 'kriminologi',
 'negeripingat',
 'melingkungi',
 'terlajak',
 'wooi',
 'penguin',
 'agung']

In [29]:
# def cleaning(word):
#     c = re.sub(r'^-+|-+$', '-', word.lower()).strip().replace('"', ' ').replace('\'', ' ')
#     c = re.sub(r'[ ]+', ' ', c).strip()
#     if len(c) and c[-1] in '-':
#         c = c[:-1]
#         c = re.sub(r'[ ]+', ' ', c).strip()
#     if len(c) and c[0] in '-':
#         c = c[1:]
#         c = re.sub(r'[ ]+', ' ', c).strip()
#     return c

# left_, right_ = [], []
# for i in range(len(left)):
#     c_l = cleaning(left[i])
#     if len(c_l) > 1 and c_l[-1] not in '-' and c_l[0] not in '-' and ' ' not in c_l and c_l.count('-') <= 1 and all([len(a) > 1 for a in c_l.split('-')]):
#         left_.append(c_l)
#     c_r = cleaning(right[i])
#     if len(c_r) > 1 and c_r[-1] not in '-' and c_r[0] not in '-' and ' ' not in c_r and c_r.count('-') <= 1 and all([len(a) > 1 for a in c_l.split('-')]):
#         right_.append(c_r)

In [30]:
# sorted_ = sorted(list(set(left_ + right_)))

In [31]:
# with open('unique-words.json', 'w') as fopen:
#     json.dump(sorted_, fopen)

In [32]:
def cleaning(word):
    c = re.sub(r'^-+|-+$', '-', word).strip().replace('"', ' ').replace('\'', ' ')
    c = re.sub(r'[ ]+', ' ', c).strip()
    if len(c) and c[-1] in '-':
        c = c[:-1]
        c = re.sub(r'[ ]+', ' ', c).strip()
    if len(c) and c[0] in '-':
        c = c[1:]
        c = re.sub(r'[ ]+', ' ', c).strip()
    return c

In [33]:
for i in tqdm(range(len(left))):
    left[i] = cleaning(left[i])

100%|█████████████████████████████████████████████████████████████████████████████| 1855109/1855109 [00:02<00:00, 671105.96it/s]


In [34]:
combined = []
for i in tqdm(range(len(left))):
    if ' ' in left[i]:
        continue
    if len(left[i]) < 2:
        continue
    if len(right[i]) < 2:
        continue
    if left[i].count('-') > 1:
        continue
    if right[i][0] in '-.':
        continue
    if right[i][-1] in '-.':
        continue
    combined.append(f'{left[i]}<<>>{right[i]}')

100%|████████████████████████████████████████████████████████████████████████████| 1855109/1855109 [00:01<00:00, 1710113.77it/s]


In [35]:
filtered_left, filtered_right = [], []
for c in set(combined):
    if '<<>>' in c:
        l, r = c.split('<<>>')
        filtered_left.append(l)
        filtered_right.append(r)

In [37]:
len(filtered_left), len(combined), len(left)

(1228060, 1796170, 1855109)

In [38]:
train_X, test_X, train_Y, test_Y = train_test_split(filtered_left, filtered_right, test_size = 0.1)

In [39]:
with open('train_stem.json', 'w') as fopen:
    json.dump({'X': train_X, 'Y': train_Y}, fopen)

In [40]:
with open('test_stem.json', 'w') as fopen:
    json.dump({'X': test_X, 'Y': test_Y}, fopen)

In [34]:
not_same = []
for i in tqdm(range(len(left))):
    left_ = left[i].lower()
    left_ = re.sub(r'^-+|-+$', '-', left_).strip().replace('"', ' ').replace('\'', ' ')
    left_ = re.sub(r'[ ]+', ' ', left_).strip()
    right_ = right[i].lower()
    
    if left_ != right_ and left[i].lower() != right[i].lower():
        left_ = cleaning(left[i])
        not_same.append((left_, right[i]))
        
len(not_same), len(not_same) / len(left)

100%|█████████████████████████████████████████████████████████████████████████████| 1855109/1855109 [00:03<00:00, 608726.77it/s]


(198320, 0.10690476947715741)

In [35]:
not_same[-1000:]

[('dilorongkan', 'lorong'),
 ('tertonggek', 'tonggek'),
 ('terkepak-kepak', 'kepak'),
 ('terungkap-ungkap', 'ungkap'),
 ('segaya', 'gaya'),
 ('menereluh', 'ruh'),
 ('empang-empang', 'empang'),
 ('dialang-alang', 'alang'),
 ('mencukup-cukupkan', 'cukup'),
 ('perwatasan', 'watas'),
 ('ringkikan', 'ringkik'),
 ('mengorek-ngorek', 'korek'),
 ('dengkuran', 'dengkur'),
 ('bertakik-takik', 'takik'),
 ('berloncat-loncatan', 'loncat'),
 ('berdahak', 'dahak'),
 ('alang-alangan', 'alang'),
 ('dirangkup', 'rangkup'),
 ('pauh-pauh', 'pauh'),
 ('dikocakkan', 'kocak'),
 ('rontaan', 'ronta'),
 ('pengerip', 'kerip'),
 ('kauseri', 'seri'),
 ('setandang', 'tandang'),
 ('penconteng', 'conteng'),
 ('meninggi-ninggikan', 'tinggi'),
 ('berrang', 'rang'),
 ('keikliman', 'iklim'),
 ('disebari', 'bari'),
 ('telatak', 'tatak'),
 ('membetot', 'betot'),
 ('berjuluran', 'julur'),
 ('dipertamakan', 'pertama'),
 ('menggertukkan', 'gertuk'),
 ('kempaan', 'kempa'),
 ('terkeluani', 'keluan'),
 ('kesetan', 'keset'),
 ('s

In [36]:
for i in range(10):
    print(i, augment_left_right('mengenjak-enjak', 'enjak'))

0 (True, 'meeengenjak-enjak', 'enjak')
1 (True, 'menggenjak-enjak', 'enjak')
2 (True, 'mnnnggenjak-enjak', 'enjak')
3 (True, 'mnnngggenjak-enjak', 'enjak')
4 (True, 'mmeenngenjak-enjak', 'enjak')
5 (True, 'mmenngenjak-enjak', 'enjak')
6 (True, 'mngenjak-enjak', 'enjak')
7 (True, 'mmeeenggenjak-enjak', 'enjak')
8 (True, 'mmmengenjak-enjak', 'enjak')
9 (True, 'mngenjak-enjak', 'enjak')


In [37]:
'<<>>'.join(augment_left_right('mengenjak-enjak', 'enjak')[1:])

'mngenjak-enjak<<>>enjak'

In [38]:
for i in range(10):
    print(i, '<<>>'.join(augment_left_right('membidai', 'bidai', augment_hubungan = False, 
                                           p_augment_word = 0.85)[1:]))

0 membidai<<>>bidai
1 membidai<<>>bidai
2 membidai<<>>bidai
3 membidai<<>>bidai
4 membidai<<>>bidai
5 membidai<<>>bidai
6 membidai<<>>bidai
7 membidai<<>>bidai
8 membidai<<>>bidai
9 membidai<<>>bidai


In [39]:
for n in tqdm(not_same):
    for _ in range(3):
        try:
            
            l = augment_left_right(n[0], n[1], augment_hubungan = False, p_augment_word = 0.85)
            if l[0]:
                combined.append('<<>>'.join(l[1:]))
        except:
            pass
        
    for _ in range(7):
        try:
            l = augment_left_right(*n)
            if l[0]:
                combined.append('<<>>'.join(l[1:]))
        except:
            pass

100%|████████████████████████████████████████████████████████████████████████████████| 198320/198320 [00:11<00:00, 17843.16it/s]


In [40]:
len(set(combined))

1932509

In [41]:
combined[-1000:]

['terrkrkau<<>>krkau',
 'tterkerkau<<>>kerkau',
 'terrkerkau<<>>kerkau',
 'terkrkau<<>>krkau',
 'teerrkerkau<<>>kerkau',
 'trkerkau<<>>kerkau',
 'mnnngurut-urut<<>>urut',
 'meengurut-urut<<>>urut',
 'mmmenngggurut-urut<<>>urut',
 'mengggurut-urut<<>>urut',
 'hadap-hadapnn<<>>hadap',
 'memperdahsyt<<>>dahsyt',
 'meeemmmprdahsyat<<>>dahsyat',
 'meemmmpperrrdahsyat<<>>dahsyat',
 'mmmemperrrdhsyat<<>>dhsyat',
 'mmmmpprdahsyat<<>>dahsyat',
 'mmmprdahsyat<<>>dahsyat',
 'mmmemmpprdahsyat<<>>dahsyat',
 'mmperdhsyat<<>>dhsyat',
 'diicita-cita<<>>cita',
 'diiicita-cita<<>>cita',
 'ddicita-cita<<>>cita',
 'dddcita-cita<<>>cita',
 'ddiicita-cita<<>>cita',
 'kemling<<>>ling',
 'kkmling<<>>ling',
 'kemling<<>>ling',
 'kemeeeling<<>>ling',
 'kmeling<<>>ling',
 'kkmling<<>>ling',
 'kewatanannn<<>>watan',
 'kewtnannn<<>>wtn',
 'keewtanaaan<<>>wtan',
 'keewatanan<<>>watan',
 'kkeeewatanan<<>>watan',
 'keeewtannnn<<>>wtan',
 'seekubin<<>>kubin',
 'ssskbn<<>>kbn',
 'sseekubin<<>>kubin',
 'seekubin<<>>kubi

In [42]:
filtered_left, filtered_right = [], []
for c in combined:
    if '<<>>' in c:
        l, r = c.split('<<>>')
        filtered_left.append(l)
        filtered_right.append(r)

In [50]:
train_X, test_X, train_Y, test_Y = train_test_split(filtered_left, filtered_right, test_size = 0.1)

In [51]:
train_X[:10], train_Y[:10]

(['muntefering',
  'raffaela',
  'Zeus',
  'kbermimpi',
  'pavyow',
  'tenangnyya',
  'mmnnjelaskn',
  'sungson',
  'Kitatakaki',
  'Proudhon'],
 ['muntefering',
  'raffaela',
  'zeus',
  'mimpi',
  'pavyow',
  'tenang',
  'jelas',
  'sungson',
  'kitatakaki',
  'proudhon'])

In [52]:
with open('train_noisy_stem.json', 'w') as fopen:
    json.dump({'X': train_X, 'Y': train_Y}, fopen)

In [53]:
with open('test_noisy_stem.json', 'w') as fopen:
    json.dump({'X': test_X, 'Y': test_Y}, fopen)

In [54]:
len(train_X), len(test_X)

(2423639, 269294)

In [44]:
with open('text.txt', 'w') as fopen:
    fopen.write('\n'.join(filtered_left + filtered_right))

In [42]:
import youtokentome as yttm

In [45]:
stemmer_bpe = yttm.BPE.train(data='text.txt', vocab_size=1000, model='stemmer-v2.model',
              pad_id=0, unk_id=2, bos_id=3, eos_id=1)

Training parameters
  input: text.txt
  model: stemmer-v2.model
  vocab_size: 1000
  n_threads: 8
  character_coverage: 1
  pad: 0
  unk: 2
  bos: 3
  eos: 1

reading file...
learning bpe...
number of unique characters in the training data: 57
number of deleted characters: 0
number of unique characters left: 57
model saved to: stemmer-v2.model
